In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44640
2,Huelva,Confirmados PDIA 14 días,875
3,Huelva,Tasa PDIA 14 días,"170,5087982539899"
4,Huelva,Confirmados PDIA 7 días,304
5,Huelva,Tasa PDIA 7 dias,"59,23962819338621"
6,Huelva,Total Confirmados,44839
7,Huelva,Curados,40266
8,Huelva,Fallecidos,405


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44640.0


/tmp/ipykernel_4088/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12379.0


/tmp/ipykernel_4088/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4088/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4088/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4088/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2283 personas en los últimos 7 días 

Un positivo PCR cada 738 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44640.0,304.0,875.0,513170.0,59.239628,170.508798,52.0
Huelva-Costa,26380.0,192.0,547.0,289548.0,66.310249,188.915137,36.0
Condado-Campiña,13895.0,79.0,234.0,156231.0,50.566149,149.778213,12.0
Huelva (capital),12379.0,63.0,195.0,143837.0,43.799579,135.570125,10.0
Cartaya,1878.0,29.0,80.0,20083.0,144.400737,398.346861,8.0
Isla Cristina,3063.0,29.0,69.0,21393.0,135.558360,322.535409,7.0
Villablanca,281.0,9.0,15.0,2885.0,311.958406,519.930676,5.0
Sierra de Huelva-Andévalo Central,3948.0,27.0,84.0,67391.0,40.064697,124.645724,4.0
Beas,358.0,1.0,10.0,4341.0,23.036167,230.361668,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,31.0,2.0,5.0,396.0,505.050505,1262.626263,1.0
San Silvestre de Guzmán,24.0,3.0,7.0,614.0,488.599349,1140.065147,0.0
San Bartolomé de la Torre,376.0,7.0,22.0,3761.0,186.120713,584.950811,1.0
Villablanca,281.0,9.0,15.0,2885.0,311.958406,519.930676,5.0
Cerro de Andévalo (El),195.0,4.0,10.0,2327.0,171.895144,429.737860,0.0
Lucena del Puerto,362.0,2.0,13.0,3261.0,61.330880,398.650721,0.0
Cartaya,1878.0,29.0,80.0,20083.0,144.400737,398.346861,8.0
Higuera de la Sierra,62.0,4.0,5.0,1291.0,309.837335,387.296669,0.0
Isla Cristina,3063.0,29.0,69.0,21393.0,135.558360,322.535409,7.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Palma del Condado (La),1412.0,2.0,22.0,10801.0,18.516804,203.684844,0.0,0.090909
Beas,358.0,1.0,10.0,4341.0,23.036167,230.361668,2.0,0.100000
Aljaraque,1608.0,3.0,26.0,21474.0,13.970383,121.076651,0.0,0.115385
Lucena del Puerto,362.0,2.0,13.0,3261.0,61.330880,398.650721,0.0,0.153846
Villanueva de los Castillejos,206.0,1.0,6.0,2825.0,35.398230,212.389381,0.0,0.166667
Valverde del Camino,831.0,2.0,11.0,12750.0,15.686275,86.274510,1.0,0.181818
Aracena,665.0,3.0,13.0,8255.0,36.341611,157.480315,1.0,0.230769
Niebla,357.0,1.0,4.0,4116.0,24.295432,97.181730,0.0,0.250000
San Bartolomé de la Torre,376.0,7.0,22.0,3761.0,186.120713,584.950811,1.0,0.318182
